In [14]:
#base_dir = '/home/gbiagini/dev/nn-sero-pytorch/'
base_dir = './'

import pandas as pd
import numpy as np
from fastai import *
from fastai.tabular import *
#from parse import _file_handler
from tqdm import tqdm

In [15]:


# _file_handler()

#loci = ['A', 'B', 'C', 'DPB1', 'DQB1', 'DRB1']
loci = ["A"]
epoch = { "A":26, "B":15, "C":13, "DPB1":13, "DQB1":20, "DRB1":13 }
layer = { "A":[1500], "B":[2500], "C":[1500], "DPB1":[1500], "DQB1":[2000], "DRB1":[1500] }


for locus in loci:
  
  AAs = []
  #tng_df = pd.read_csv(base_dir + 'RSNNS_fixed/training/' + locus + '_train.csv')
  #tst_df = pd.read_csv(base_dir + 'RSNNS_fixed/testing/' + locus + '_test.csv')
  #val_df = pd.read_csv(base_dir + 'RSNNS_fixed/training/' + locus + '_validation.csv')
  tng_df = pd.read_csv(base_dir + 'old_sets/train/' + locus + '_train.csv')
  tst_df = pd.read_csv(base_dir + 'old_sets/test/' + locus + '_test.csv')
  val_df = pd.read_csv(base_dir + 'old_sets/train/' + locus + '_validation.csv')
  tng_idx = len(tng_df)
  val_idx = len(val_df) + 1
  tst_idx = len(tst_df)
  tbatch = tng_idx // 2
  if (tbatch <= 1):
    tbatch = tng_idx // 2
  vbatch = val_idx // 2
  if (vbatch <= 1):
    vbatch = val_idx // 2
  
  df = tng_df.append(val_df)

  for each in tng_df:
    if (each != 'allele') & (each != 'serology'):
      AAs.append(each)

  dep_var = 'serology'
  cat_names = ['allele'] + AAs
  #procs = [FillMissing, Categorify]


  cat_names = ['allele'] + AAs
  test = TabularList.from_df(tst_df, path=Path(''), cat_names=cat_names)
  
  data = (TabularList.from_df(df=df, path=Path(''), procs=procs, cat_names=cat_names)
                              .split_by_idx(list(range(tng_idx,val_idx)))
                              .label_from_df(cols=dep_var, label_delim=';')
                              .add_test(test)
                              .databunch(bs=tbatch, val_bs=vbatch))
  '''
  data = (TabularList.from_df(df=df, path=Path(''), procs=procs, cat_names=cat_names)
                              .split_by_idx(list(range(tng_idx,val_idx)))
                              .label_from_df(cols=dep_var, label_delim=';')
                              .add_test(test)
                              .databunch(bs=tng_idx, val_bs=val_idx))
  '''
  acc_02 = partial(accuracy_thresh, thresh=0.99)
  f_score = partial(fbeta, thresh=0.51)

  pre_vote = {}
  avg_pred = {}
  all_models = []

  for n in range(1,11):
    learn = tabular_learner(data, opt_func = optim.SGD, layers=layer[locus], metrics=[acc_02, f_score])
    print(data.classes)

    lr = 0.5
    #learn.recorder.plot(suggestion=True)

    #learn.fit(epoch[locus], lr)
    #learn.fit_one_cycle(epoch[locus], lr)
    learn.fit_one_cycle(epoch[locus], max_lr=slice(lr))
    learn.model
    #learn.recorder.plot_losses()

    test_id = list(tst_df['allele'])

    classes = data.classes
    predictions = []
    print(classes)

   
    for i in tqdm(range(0,tst_idx)):
      category = str(learn.predict(tst_df.iloc[i], thresh=0.51)[0])
      sero = category.strip('MultiCategory ')
      sero = sero.replace(';',' ')
      sero = sero.replace('a','')
      predictions.append(sero.split())
    

    pre_vote = {test_id[j]: str(predictions[j]) for j in range(len(test_id)) }
    avg_pred[str(n)] = pre_vote
    

  avg_frame = pd.DataFrame.from_dict(avg_pred)
  mode = avg_frame.mode(axis=1)
  rmode = mode[0]
  rmode = pd.DataFrame(rmode)
  rmode = rmode.reset_index()
  rmode.columns=['allele', 'serology']

  #output_preds = pd.DataFrame({'allele': test_id, 'serology': predictions})
  rmode.to_csv(base_dir + 'predictions/' + locus + '_predictions.csv', index=False)


NameError: name 'TabularList' is not defined

In [129]:
  loci = ['A', 'B', 'C', 'DPB1', 'DQB1', 'DRB1']
#loci = ['A']

# function to check if value can be an integer - to eliminate excess characters from serology labels
def checkInt(x):
    try:
        int(x)
        return True
    except ValueError:
        return False


for loc in loci:
    newDict = {}
    simDict = {}
    diffDict = {}
    oldPredict = {}
    newPredict = {}
    oldPredFile = base_dir + "old-predictions/" + loc + ".chile"
    newPreds = pd.read_csv(base_dir + "predictions/" + loc + "_predictions.csv")
    newPreds = newPreds.set_index('allele')
    newPreds = newPreds.to_dict()
    newPredict = newPreds["serology"]
    for nKey in newPredict.keys():
        adjustMe = newPredict[nKey]
        adjustMe = adjustMe.replace('[','')
        adjustMe = adjustMe.replace(']','')
        adjustMe = adjustMe.replace(' ','')
        adjustMe = adjustMe.replace("'",'')
        adjustMe = adjustMe.split(',')
        newPredict[nKey] = [x.strip('a') for x in adjustMe if checkInt(x)]
    with open(oldPredFile, "r") as handle:
        for line in handle:
            if line.find('%') == -1:
              next
            else:
                line = line.split()
                if line == []:
                    next
                else:
                    line[:] = [x for x in line if (x != '[100.00%]')]
                    allele = loc + "*" + str(line[0][:-1])
                    oldPredict[allele] = line[1:]


    for each in oldPredict.keys():
        allDict = {}
        allDict["Allele"] = each
        allDict["Old Assignment"] = oldPredict[each]
        allDict["New Assignment"] = newPredict[each]
        if each not in newPredict.keys():
          next
        elif set(newPredict[each]) != set(oldPredict[each]):
            diffDict[each] = allDict
        elif set(newPredict[each]) == set(oldPredict[each]):
            simDict[each] = allDict
    diffFrame = pd.DataFrame.from_dict(diffDict)
    diffFrame = diffFrame.transpose()
    diffFrame.to_csv(base_dir + "comparison/" + loc + "_compfile.csv", index=False)
    simFrame = pd.DataFrame.from_dict(simDict)
    simFrame = simFrame.transpose()
    simFrame.to_csv(base_dir + "comparison/" + loc + "_similar.csv", index=False)
    

    for allele in newPredict.keys():
        allDict = {}
        allDict["Allele"] = allele
        allDict["Serologic Assignment"] = newPredict[allele]
        if allele not in oldPredict.keys():
            newDict[allele] = allDict
    newFrame = pd.DataFrame.from_dict(simDict)
    newFrame = newFrame.transpose()
    newFrame.to_csv(base_dir + "comparison/" + loc + "_newsies.csv", index=False)

    simLen = len(simFrame)
    diffLen = len(diffFrame)
    print("HLA-" +loc+ " Similar: " + str(simLen))
    print("HLA-" +loc+ " Different: " + str(diffLen))

    concordance = (simLen / (simLen + diffLen)) * 100
    print("HLA-" +loc+ " Concordance: " + str(concordance) + "%")


HLA-A Similar: 1620
HLA-A Different: 72
HLA-A Concordance: 95.74468085106383%
HLA-B Similar: 1776
HLA-B Different: 496
HLA-B Concordance: 78.16901408450704%
HLA-C Similar: 878
HLA-C Different: 443
HLA-C Concordance: 66.46479939439818%
HLA-DPB1 Similar: 141
HLA-DPB1 Different: 16
HLA-DPB1 Concordance: 89.80891719745223%
HLA-DQB1 Similar: 245
HLA-DQB1 Different: 31
HLA-DQB1 Concordance: 88.76811594202898%
HLA-DRB1 Similar: 876
HLA-DRB1 Different: 128
HLA-DRB1 Concordance: 87.25099601593625%
